In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [10]:
TARGET_LEAGUES = [
    'LTC', 'LCK',
    'LPL',
    'EU LCS', 'LEC',
    'NA LCS', 'LCS',
    'GPL', 'LST',
    'LLA', 'CLS', 'LLN',
    'LCO', 'OPL',
    'LCL',
    'LJL',
    'LMS', 'PCS',
    'VCS',
    'CBLOL', 
    'TCL',
    
    'MSI',
    'WCS',
]

In [12]:
def get_new_id(teams):
    id_list = sorted(teams['team_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

In [13]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [14]:
lst = []
for league in TARGET_LEAGUES:
    if league not in leagues['League Short'].values:
        lst.append(league)
lst

[]

In [15]:
teams = pd.read_csv('./csv/teams_id.csv')

for league in TARGET_LEAGUES:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        scoreboard_games = get_scoreboard_games(
            where=f'T.OverviewPage="{page}"'
        )
        if scoreboard_games is not None:
            scoreboard_games = scoreboard_games.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
    if len(names) > 0:
        print(scoreboard_games['OverviewPage'].unique()[0])
        print(names)
        break

LTC
                                 Name  \
0   Azubu LoL The Champions All Stars   
1               Champions 2012 Spring   
2               Champions 2012 Summer   
3               Champions 2013 Spring   
4    Champions 2013 Spring Qualifiers   
5               Champions 2013 Summer   
6    Champions 2013 Summer Qualifiers   
7               Champions 2013 Winter   
8    Champions 2013 Winter Qualifiers   
9               Champions 2014 Spring   
10   Champions 2014 Spring Qualifiers   
11              Champions 2014 Summer   
12   Champions 2014 Summer Qualifiers   
13              Champions 2014 Winter   
14   Champions 2014 Winter Qualifiers   
15              Champions 2015 Spring   
16     Champions 2015 Spring Playoffs   
17    Champions 2015 Spring Preseason   
18              Champions 2015 Summer   
19     Champions 2015 Summer Playoffs   
20    Champions 2015 Summer Promotion   
21   Champions Spring 2015 Qualifiers   

                               OverviewPage  DateSta

In [518]:
teams.loc[teams['team'].str.contains('Afro')]

,team,team_id
531,Team Afro,587


In [529]:
teams = pd.concat(
    [
        teams,
        pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
        # pd.Series({'team': names.pop(0), 'team_id': 587}).to_frame().T
    ],
    ignore_index=True
)
print(names)
teams

[]


,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
694,paiN Gaming,262
695,team propertybb2,139
696,vVv Gaming,213
697,yoe Flash Wolves,459


In [530]:
names

[]

In [531]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
694,n!faculty,168
695,paiN Gaming,262
696,team propertybb2,139
697,vVv Gaming,213


In [532]:
teams.to_csv('teams_id.csv', index=False)

In [579]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [16]:
for year in range(2011, 2023):
    tournaments = pd.DataFrame()
    for league in TARGET_LEAGUES:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape)

2011 (1, 17)
2012 (6, 17)
2013 (27, 17)
2014 (49, 17)
2015 (74, 17)
2016 (74, 17)
2017 (86, 17)
2018 (84, 17)
2019 (86, 17)
2020 (61, 17)
2021 (61, 17)
2022 (59, 17)


In [17]:
for year in range(2011, 2023):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    print(year, tournaments.shape)
    scoreboard_games = pd.DataFrame()
    for page in tournaments['OverviewPage']:
        sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
        if sg is None:
            print(f'{year}\t{page}\tdrop')
            tournaments.drop(tournaments.loc[tournaments['OverviewPage'] == page].index, inplace=True)
            continue
        league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
        league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
        sg['League'] = league
        sg = sg.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        print(f'\t{sg.shape}\t{page}')
        scoreboard_games = pd.concat([scoreboard_games, sg])
    scoreboard_games.to_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv', index=False)
    print(year, scoreboard_games.shape)
    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape, '\n\n')

2011 (1, 17)
	(28, 37)	Season 1 World Championship
2011 (28, 37)
2011 (1, 17) 


2012 (6, 17)
	(44, 37)	Champions/2012 Season/Spring
	(46, 37)	Champions/2012 Season/Summer
2012	Season 2/Regional Finals/China	drop
	(17, 37)	Season 2/Regional Finals/Europe
	(16, 37)	Season 2/Regional Finals/Korea
	(31, 37)	Season 2 World Championship
2012 (154, 37)
2012 (5, 17) 


2013 (27, 17)
2013	Champions/2013 Season/Winter Qualifiers	drop
	(112, 37)	Champions/2013 Season/Winter
2013	NA LCS/Season 3/Spring Qualifiers	drop
2013	LPL/2013 Season/Spring Qualifier	drop
	(112, 37)	NA LCS/Season 3/Spring Season
	(112, 37)	EU LCS/Season 3/Spring Season
2013	Champions/2013 Season/Spring Qualifiers	drop
	(112, 37)	LPL/2013 Season/Spring Season
	(87, 37)	Champions/2013 Season/Spring
	(15, 37)	EU LCS/Season 3/Spring Playoffs
	(20, 37)	NA LCS/Season 3/Spring Playoffs
	(28, 37)	EU LCS/Season 3/Summer Promotion
	(24, 37)	NA LCS/Season 3/Summer Promotion
2013	Season 3 Taiwan Regional Finals/Hong Kong Qualifiers	drop